process files in a subfolder (needs new path for each case)

In [ ]:
import openslide
from openslide import open_slide
import openslide
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
import openslide
from openslide import open_slide
import openslide
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from openslide import open_slide
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff
from openslide.deepzoom import DeepZoomGenerator
import os
def norm_HnE(img, Io=240, alpha=1, beta=0.15):


    ######## Step 1: Convert RGB to OD ###################
    ## reference H&E OD matrix.
    #Can be updated if you know the best values for your image. 
    #Otherwise use the following default values. 
    #Read the above referenced papers on this topic. 
    HERef = np.array([[0.5626, 0.2159],
                      [0.7201, 0.8012],
                      [0.4062, 0.5581]])
    ### reference maximum stain concentrations for H&E
    maxCRef = np.array([1.9705, 1.0308])
    
    
    # extract the height, width and num of channels of image
    h, w, c = img.shape
    
    # reshape image to multiple rows and 3 columns.
    #Num of rows depends on the image size (wxh)
    img = img.reshape((-1,3))
    
    # calculate optical density
    # OD = −log10(I)  
    #OD = -np.log10(img+0.004)  #Use this when reading images with skimage
    #Adding 0.004 just to avoid log of zero. 
    
    OD = -np.log10((img.astype(np.float)+1)/Io) #Use this for opencv imread
    #Add 1 in case any pixels in the image have a value of 0 (log 0 is indeterminate)
    
    
    ############ Step 2: Remove data with OD intensity less than β ############
    # remove transparent pixels (clear region with no tissue)
    ODhat = OD[~np.any(OD < beta, axis=1)] #Returns an array where OD values are above beta
    #Check by printing ODhat.min()
    
    ############# Step 3: Calculate SVD on the OD tuples ######################
    #Estimate covariance matrix of ODhat (transposed)
    # and then compute eigen values & eigenvectors.
    eigvals, eigvecs = np.linalg.eigh(np.cov(ODhat.T))
    
    
    ######## Step 4: Create plane from the SVD directions with two largest values ######
    #project on the plane spanned by the eigenvectors corresponding to the two 
    # largest eigenvalues    
    That = ODhat.dot(eigvecs[:,1:3]) #Dot product
    
    ############### Step 5: Project data onto the plane, and normalize to unit length ###########
    ############## Step 6: Calculate angle of each point wrt the first SVD direction ########
    #find the min and max vectors and project back to OD space
    phi = np.arctan2(That[:,1],That[:,0])
    
    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100-alpha)
    
    vMin = eigvecs[:,1:3].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
    vMax = eigvecs[:,1:3].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)
    
    
    # a heuristic to make the vector corresponding to hematoxylin first and the 
    # one corresponding to eosin second
    if vMin[0] > vMax[0]:    
        HE = np.array((vMin[:,0], vMax[:,0])).T
        
    else:
        HE = np.array((vMax[:,0], vMin[:,0])).T
    
    
    # rows correspond to channels (RGB), columns to OD values
    Y = np.reshape(OD, (-1, 3)).T
    
    # determine concentrations of the individual stains
    C = np.linalg.lstsq(HE,Y, rcond=None)[0]
    
    # normalize stain concentrations
    maxC = np.array([np.percentile(C[0,:], 99), np.percentile(C[1,:],99)])
    tmp = np.divide(maxC,maxCRef)
    C2 = np.divide(C,tmp[:, np.newaxis])
    
    ###### Step 8: Convert extreme values back to OD space
    # recreate the normalized image using reference mixing matrix 
    
    Inorm = np.multiply(Io, np.exp(-HERef.dot(C2)))
    Inorm[Inorm>255] = 254
    Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)  
    
    # Separating H and E components
    
    H = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,0], axis=1).dot(np.expand_dims(C2[0,:], axis=0))))
    H[H>255] = 254
    H = np.reshape(H.T, (h, w, 3)).astype(np.uint8)
    
    E = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,1], axis=1).dot(np.expand_dims(C2[1,:], axis=0))))
    E[E>255] = 254
    E = np.reshape(E.T, (h, w, 3)).astype(np.uint8)
    
    return (Inorm, H, E)
    
for root, dirs, files in os.walk(os.path.abspath(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\TCGA-4P-AA8J")):
    for file in files:
        print(os.path.join(root, file))
        pathi = os.path.join(root, file)
        slide = open_slide(pathi)
        pathi2= os.path.dirname(pathi)
        #slide = open_slide(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\TCGA-4P-AA8J\TCGA-4P-AA8J-01A-01-TS1.D62B960E-A357-4E3D-9CB5-93F112174546.svs")

        #Generate object for tiles using the DeepZoomGenerator
        tiles = DeepZoomGenerator(slide, tile_size=1000, overlap=0, limit_bounds=False)
        #Here, we have divided our svs into tiles of size 256 with no overlap. 

        #The tiles object also contains data at many levels. 
        #To check the number of levels
        print("The number of levels in the tiles object are: ", tiles.level_count)
        print("The dimensions of data in each level are: ", tiles.level_dimensions)
        #Total number of tiles in the tiles object
        print("Total number of tiles = : ", tiles.tile_count)

        level_count_number = tiles.level_count - 1
        ###### processing and saving each tile to local directory
        cols, rows = tiles.level_tiles[level_count_number]


        #orig_tile_dir_name = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\original_tiles"
        norm_tile_dir_name = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\normalized_tiles"
        #H_tile_dir_name = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\H_tiles"
        #E_tile_dir_name = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\E_tiles"

        # if destination dir does not exists, create it
        #if os.path.isdir(orig_tile_dir_name) == False:
                #os.makedirs(orig_tile_dir_name)

        # if destination dir does not exists, create it
        if os.path.isdir(norm_tile_dir_name) == False:
                os.makedirs(norm_tile_dir_name)

        # if destination dir does not exists, create it
        #if os.path.isdir(H_tile_dir_name) == False:
                #os.makedirs(H_tile_dir_name)

        # if destination dir does not exists, create it
        #if os.path.isdir(E_tile_dir_name) == False:
                #os.makedirs(E_tile_dir_name)

        for row in range(rows):
            for col in range(cols):
                tile_name = str(col) + "_" + str(row)
                #tile_name = os.path.join(tile_dir, '%d_%d' % (col, row))
                #print("Now processing tile with title: ", tile_name)
                temp_tile = tiles.get_tile(level_count_number, (col, row))
                temp_tile_RGB = temp_tile.convert('RGB')
                temp_tile_np = np.array(temp_tile_RGB)
                #Save original tile
                #tiff.imsave(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\original_tiles/"+tile_name + "_original.tif", temp_tile_np)
        
                if temp_tile_np.mean() < 200:
                    print("Processing tile number:", tile_name)
                    norm_img, H_img, E_img = norm_HnE(temp_tile_np, Io=240, alpha=1, beta=0.15)
                #Save the norm tile, H and E tiles      
            
                    tiff.imsave(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\normalized_tiles/"+tile_name + "_norm.tif", norm_img)
                    #tiff.imsave(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\H_tiles/"+tile_name + "_H.tif", H_img)
                    #tiff.imsave(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\E_tiles/" +tile_name + "_E.tif", E_img)
            
                else:
                    print("NOT PROCESSING TILE:", tile_name)
from PIL import Image
import numpy as np
import os

path = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\normalized_tiles/"
# Delete images with the high pixel value
for filename in os.listdir(path):
    images = Image.open(os.path.join(path,filename))
    width, height = images.size
    print(images.size)
    if images.size != (1000, 1000):
        images.close()
        os.remove(os.path.join(path, filename))
images.close()

import os
import random
import shutil


files_list = []

for root, dirs, files in os.walk(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\normalized_tiles"):
    for file in files:
        #all 
        if file.endswith(".tif"):
            files_list.append(os.path.join(root, file))


#print images
#lets me count and print the amount of jpeg,jpg,pmg 
file_count = len(files_list)
print (file_count)

# print files_list   
filesToCopy = random.sample(files_list, 10)  #prints two random files from list 

destPath = pathi2

# if destination dir does not exists, create it
if os.path.isdir(destPath) == False:
        os.makedirs(destPath)

# iteraate over all random files and move them
for file in filesToCopy:
    shutil.move(file, destPath)

source = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\normalized_tiles"
destination = pathi2
dest = shutil.move(source, destination) 

#source = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\H_tiles"
#destination = pathi2
#dest = shutil.move(source, destination) 

#source = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\E_tiles"
#destination = pathi2
#dest = shutil.move(source, destination) 

#source = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\original_tiles"
#destination = pathi2
#dest = shutil.move(source, destination) 


Process all subfolders and files within it in a root folder

In [ ]:
import openslide
from openslide import open_slide
import openslide
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
import openslide
from openslide import open_slide
import openslide
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from openslide import open_slide
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff
from openslide.deepzoom import DeepZoomGenerator
import os
def norm_HnE(img, Io=240, alpha=1, beta=0.15):


    ######## Step 1: Convert RGB to OD ###################
    ## reference H&E OD matrix.
    #Can be updated if you know the best values for your image. 
    #Otherwise use the following default values. 
    #Read the above referenced papers on this topic. 
    HERef = np.array([[0.5626, 0.2159],
                      [0.7201, 0.8012],
                      [0.4062, 0.5581]])
    ### reference maximum stain concentrations for H&E
    maxCRef = np.array([1.9705, 1.0308])
    
    
    # extract the height, width and num of channels of image
    h, w, c = img.shape
    
    # reshape image to multiple rows and 3 columns.
    #Num of rows depends on the image size (wxh)
    img = img.reshape((-1,3))
    
    # calculate optical density
    # OD = −log10(I)  
    #OD = -np.log10(img+0.004)  #Use this when reading images with skimage
    #Adding 0.004 just to avoid log of zero. 
    
    OD = -np.log10((img.astype(np.float)+1)/Io) #Use this for opencv imread
    #Add 1 in case any pixels in the image have a value of 0 (log 0 is indeterminate)
    
    
    ############ Step 2: Remove data with OD intensity less than β ############
    # remove transparent pixels (clear region with no tissue)
    ODhat = OD[~np.any(OD < beta, axis=1)] #Returns an array where OD values are above beta
    #Check by printing ODhat.min()
    
    ############# Step 3: Calculate SVD on the OD tuples ######################
    #Estimate covariance matrix of ODhat (transposed)
    # and then compute eigen values & eigenvectors.
    eigvals, eigvecs = np.linalg.eigh(np.cov(ODhat.T))
    
    
    ######## Step 4: Create plane from the SVD directions with two largest values ######
    #project on the plane spanned by the eigenvectors corresponding to the two 
    # largest eigenvalues    
    That = ODhat.dot(eigvecs[:,1:3]) #Dot product
    
    ############### Step 5: Project data onto the plane, and normalize to unit length ###########
    ############## Step 6: Calculate angle of each point wrt the first SVD direction ########
    #find the min and max vectors and project back to OD space
    phi = np.arctan2(That[:,1],That[:,0])
    
    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100-alpha)
    
    vMin = eigvecs[:,1:3].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
    vMax = eigvecs[:,1:3].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)
    
    
    # a heuristic to make the vector corresponding to hematoxylin first and the 
    # one corresponding to eosin second
    if vMin[0] > vMax[0]:    
        HE = np.array((vMin[:,0], vMax[:,0])).T
        
    else:
        HE = np.array((vMax[:,0], vMin[:,0])).T
    
    
    # rows correspond to channels (RGB), columns to OD values
    Y = np.reshape(OD, (-1, 3)).T
    
    # determine concentrations of the individual stains
    C = np.linalg.lstsq(HE,Y, rcond=None)[0]
    
    # normalize stain concentrations
    maxC = np.array([np.percentile(C[0,:], 99), np.percentile(C[1,:],99)])
    tmp = np.divide(maxC,maxCRef)
    C2 = np.divide(C,tmp[:, np.newaxis])
    
    ###### Step 8: Convert extreme values back to OD space
    # recreate the normalized image using reference mixing matrix 
    
    Inorm = np.multiply(Io, np.exp(-HERef.dot(C2)))
    Inorm[Inorm>255] = 254
    Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)  
    
    # Separating H and E components
    
    H = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,0], axis=1).dot(np.expand_dims(C2[0,:], axis=0))))
    H[H>255] = 254
    H = np.reshape(H.T, (h, w, 3)).astype(np.uint8)
    
    E = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,1], axis=1).dot(np.expand_dims(C2[1,:], axis=0))))
    E[E>255] = 254
    E = np.reshape(E.T, (h, w, 3)).astype(np.uint8)
    
    return (Inorm, H, E)


tumb = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\test"

# for root, dirs, files in os.walk(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\test"):
for dirs in os.listdir(tumb):
    # if len(files) == 0:
    #     continue
    # else:
    for file in os.listdir(os.path.join(tumb, dirs)):
        pathi = os.path.join(tumb,dirs,file)

        # print(os.path.join(root, file))
        # pathi = os.path.join(root, file)
        slide = open_slide(pathi)
        pathi2= os.path.dirname(pathi)
        #slide = open_slide(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\TCGA-4P-AA8J\TCGA-4P-AA8J-01A-01-TS1.D62B960E-A357-4E3D-9CB5-93F112174546.svs")

        #Generate object for tiles using the DeepZoomGenerator
        tiles = DeepZoomGenerator(slide, tile_size=1000, overlap=0, limit_bounds=False)
        #Here, we have divided our svs into tiles of size 256 with no overlap. 

        #The tiles object also contains data at many levels. 
        #To check the number of levels
        print("The number of levels in the tiles object are: ", tiles.level_count)
        print("The dimensions of data in each level are: ", tiles.level_dimensions)
        #Total number of tiles in the tiles object
        print("Total number of tiles = : ", tiles.tile_count)

        level_count_number = tiles.level_count - 1
        ###### processing and saving each tile to local directory
        cols, rows = tiles.level_tiles[level_count_number] 


        #orig_tile_dir_name = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\original_tiles"
        norm_tile_dir_name = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\normalized_tiles"
        #H_tile_dir_name = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\H_tiles"
        #E_tile_dir_name = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\E_tiles"

        # if destination dir does not exists, create it
        #if os.path.isdir(orig_tile_dir_name) == False:
                #os.makedirs(orig_tile_dir_name)

        # if destination dir does not exists, create it
        if os.path.isdir(norm_tile_dir_name) == False:
                os.makedirs(norm_tile_dir_name)

        # if destination dir does not exists, create it
        #if os.path.isdir(H_tile_dir_name) == False:
                #os.makedirs(H_tile_dir_name)

        # if destination dir does not exists, create it
        #if os.path.isdir(E_tile_dir_name) == False:
                #os.makedirs(E_tile_dir_name)

        for row in range(rows):
            for col in range(cols):
                tile_name = str(col) + "_" + str(row)
                #tile_name = os.path.join(tile_dir, '%d_%d' % (col, row))
                #print("Now processing tile with title: ", tile_name)
                temp_tile = tiles.get_tile(level_count_number, (col, row))
                temp_tile_RGB = temp_tile.convert('RGB')
                temp_tile_np = np.array(temp_tile_RGB)
                #Save original tile
                #tiff.imsave(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\original_tiles/"+tile_name + "_original.tif", temp_tile_np)
        
                if temp_tile_np.mean() < 200:
                    print("Processing tile number:", tile_name)
                    norm_img, H_img, E_img = norm_HnE(temp_tile_np, Io=240, alpha=1, beta=0.15)
                #Save the norm tile, H and E tiles      
            
                    tiff.imsave(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\normalized_tiles/"+tile_name + "_norm.tif", norm_img)
                    #tiff.imsave(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\H_tiles/"+tile_name + "_H.tif", H_img)
                    #tiff.imsave(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\E_tiles/" +tile_name + "_E.tif", E_img)
            
                else:
                    print("NOT PROCESSING TILE:", tile_name)


    from PIL import Image
    import numpy as np
    import os

    path = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\normalized_tiles/"
    # Delete images with the high pixel value
    for filename in os.listdir(path):
        images = Image.open(os.path.join(path,filename))
        width, height = images.size
        print(images.size)
        if images.size != (1000, 1000):
            images.close()
            os.remove(os.path.join(path, filename))
    images.close()

    import os
    import random
    import shutil


    files_list = []

    for root, dirs, files in os.walk(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\normalized_tiles"):
        for file in files:
            #all 
            if file.endswith(".tif"):
                files_list.append(os.path.join(root, file))


    #print images
    #lets me count and print the amount of jpeg,jpg,pmg 
    file_count = len(files_list)
    print (file_count)

    # print files_list   
    filesToCopy = random.sample(files_list, 10)  #prints two random files from list 

    destPath = pathi2

    # if destination dir does not exists, create it
    if os.path.isdir(destPath) == False:
            os.makedirs(destPath)

    # iteraate over all random files and move them
    for file in filesToCopy:
        shutil.move(file, destPath)

    source = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\normalized_tiles"
    destination = pathi2
    dest = shutil.move(source, destination) 

    #source = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\H_tiles"
    #destination = pathi2
    #dest = shutil.move(source, destination) 

    #source = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\E_tiles"
    #destination = pathi2
    #dest = shutil.move(source, destination) 

    #source = r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\original_tiles"
    #destination = pathi2
    #dest = shutil.move(source, destination) 




In [ ]:
#import pyvips
from openslide import open_slide
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff

#Extract a small region from the large file (level 0)
#Let us extract a region from somewhere in the middle - coords 16k, 16k
#Extract 1024,1024 region
smaller_region = slide.read_region((4000,4000), 0, (1024,1024))
smaller_region_RGB = smaller_region.convert('RGB')
smaller_region_np = np.array(smaller_region_RGB)

plt.axis('off')
plt.imshow(smaller_region_np)

norm_img, H_img, E_img = norm_HnE(smaller_region_np, Io=240, alpha=1, beta=0.15)

plt.figure(figsize=(12, 12))
plt.subplot(221)
plt.title('Original Image')
plt.imshow(smaller_region_np)
plt.subplot(222)
plt.title('Normalized Image')
plt.imshow(norm_img)
plt.subplot(223)
plt.title('H image')
plt.imshow(H_img)
plt.subplot(224)
plt.title('E image')
plt.imshow(E_img)
plt.show()

In [ ]:
from openslide.deepzoom import DeepZoomGenerator
#slide = open_slide(r"C:\Users\Babak\Desktop\Pathogenomics\TCGA_data_OSCC_cases\TCGA-4P-AA8J\TCGA-4P-AA8J-01A-01-TS1.D62B960E-A357-4E3D-9CB5-93F112174546.svs")

#Generate object for tiles using the DeepZoomGenerator
tiles = DeepZoomGenerator(slide, tile_size=1000, overlap=0, limit_bounds=False)
#Here, we have divided our svs into tiles of size 256 with no overlap. 

#The tiles object also contains data at many levels. 
#To check the number of levels
print("The number of levels in the tiles object are: ", tiles.level_count)
print("The dimensions of data in each level are: ", tiles.level_dimensions)
#Total number of tiles in the tiles object
print("Total number of tiles = : ", tiles.tile_count)

Excel_Merge_code

In [ ]:
import glob
import pandas as pd
 
# specifying the path to csv files
path = r"C:\Users\Babak\Downloads\bftools\bftools\splitted"
 
# csv files in the path
file_list = glob.glob(path + "/*.csv")
 
# list of excel files we want to merge.
# pd.read_excel(file_path) reads the 
# excel data into pandas dataframe.
excl_list = []
 
for file in file_list:
    tato = pd.read_csv(file)
    tato.loc['mean'] = tato.mean()
    tato = tato.drop(tato.index.to_list()[0:-1] ,axis = 0 )
    excl_list.append(tato)

 
# concatenate all DataFrames in the list
# into a single DataFrame, returns new
# DataFrame.

excl_merged = pd.concat(excl_list)

In [ ]:
print(file_list)

In [ ]:
excl_merged.to_csv(r"C:\Users\Babak\Downloads\bftools\bftools\splitted\Bank_Stocks.csv", index=False)

In [ ]:
excl_merged.head()

In [ ]:
excl_merged.shape[0]

In [ ]:
excl_merged.loc['mean'] = excl_merged.mean()

In [ ]:
excl_merged = excl_merged.drop(excl_merged.index.to_list()[0:-1] ,axis = 0 )
